<a href="https://colab.research.google.com/github/emailic/YOLOv5-Microplasticos/blob/main/crop_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

File for cropping label files according to cropped images, previously cropped using the file crop_images.ipynb. Make sure to adjust the overlap to be one third of your desired output image size, as well as the original image size pixels in function convert_coordinates.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from os import walk, getcwd
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
mypath = '/content/drive/My Drive/microplasticos/labels/train'
outpath = "/content/drive/My Drive/microplasticos/labels/train_576"

In [ ]:
label_name_list = []
for file in os.listdir(mypath):
    label_name_list.append(file)

In [ ]:
#check if the row is present in the cut image
def isInTheImage(label_dir, row):
    n=label_dir.find('grid_')
    pcords=label_dir[(n+5):].strip('.jpg') #picture coordinates
    #print(cords)
    pcords=pcords.split('_')
    #print(cords)
    #pixwl boundaries of the image
    top=int(pcords[0])#/3456
    left=int(pcords[1])#/4608
    right= left + (192 * 3)#/4608 #before there was 384 instead of 192
    bottom= top + (192 * 3)#/3456

    bbcords=row.split()[1:] #bounding box (label) coordinates
    lbbcords=[] #list of bounding box coordinates
    for x in bbcords:
      lbbcords.append(float(x))
      
    #print('four coords', coords)
    if 3456*(lbbcords[1]+ 0.5*lbbcords[3])>top and 3456*(lbbcords[1]- 0.5*lbbcords[3])<bottom and 4608*((lbbcords[2]/2+lbbcords[0]))<right and 4608*((lbbcords[0]-lbbcords[2]/2))>left:
      return True
    else:
      #print('croppet photo top bottom right and left',top, bottom, right, left  )
      #print('top, bottom, right and left, ', 3456*(lbbcords[1]+ 0.5*lbbcords[3]), 3456*(lbbcords[1]- 0.5*lbbcords[3]), 4608*((lbbcords[2]/2+lbbcords[0])),4608*((lbbcords[0]-lbbcords[2]/2)) )
      return False
    

isInTheImage('/content/drive/My Drive/microplasticos/labels/train_1152/tar-03-olympus-10-01-2020.jpg_grid_0_0.jpg','0 0.98 0.95538580246913578 0.0001 0.001')

      
    




False

In [ ]:
def convert_coordinates(line,image): #original label directory, cropped image
    idx=image.find('grid_')
    pcords=image[(idx+5):].strip('.jpg').split('_') #picture top left  coordinates
    #pixwl boundaries of the image, with respect to the original image
    top=int(pcords[0])#/3456
    left=int(pcords[1])#/4608
    right= left + (192 * 3)#/4608 #before there was 384 instead of 192
    bottom= top + (192 * 3)#/3456
    #original image coordinates converted and normalized to the cropped picture
    line=line.split(' ')
    w=float(line[3])*8 # width/576,  so 4608/576  so 8 
    h=float(line[4])*6 # height/576
    x=(float(line[1])*4608-left-w)/576 #X
    y=(float(line[2])*3456-top-h)/576 #y
    if x<0:
      x=0
    if x>1:
      x=1
    if y<0:
      y=0
    if y>1:
      y=1
    #in case the bounding box is outside the picture
    if x-w/2<0:
      w=2*x
    if y-h/2<0:
      h=2*y
    if w/2+x>1:
      w=2*(1-x)
    if h/2+y>1:
      h=2*(1-y)
    
  
    newline=line[0]+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)
    return newline

convert_coordinates('3 0.515140892094017 0.3327991452991453 0.04173344017094018 0.06009615384615382', '20220516_113208.jpg_grid_1152_1536.jpg' )

    


'3 1 0 0 0'

In [ ]:
original=  open('/content/drive/My Drive/microplasticos/labels/train/20220516_113208.txt', "r")
#new_path=os.path.join(outpath, filename.strip('.txt') + ".jpg_grid_" + str(top) + "_" + str(left) + ".txt")
#new = open('/content/drive/My Drive/microplasticos/labels/train_1152/tar-03-olympus-10-01-2020.jpg_grid_0_3456.txt',  "a+")
lines = original.read().split('\n')
#print(lines)
#print(mypath+'/'+filename)
#print(new_path)
for line in lines:
    #print(line)
    if line!='':
        #print(new_path)
        if isInTheImage('/content/drive/My Drive/microplasticos/images/train_1152/20220516_113208.jpg_grid_384_1536.jpg', line):
            print(line)
            #new.write(line+'\n')


3 0.39912192841880334 0.18867966524216523 0.03338675213675213 0.04562856125356126
3 0.4233273237179487 0.2543402777777778 0.05174946581196573 0.06566061253561249


In [ ]:
overlap=192

grid_image = Image.open('/content/drive/My Drive/microplasticos/images/train/20220516_113208.jpg')
grid_width, grid_height = grid_image.size

for h in range(1, int(grid_height/overlap) - 1): #(1, 8)(1,17)
    for w in range(1, int(grid_width/overlap) - 1): #(1, 11) (1,23)
          left = overlap * (w - 1) 
          right = left + (overlap * 3)
          top = overlap * (h - 1) 
          bot = top + (overlap * 3)
          #print(left, top, right, bot)
          
          #crop_img = grid_image.crop((left, top, right, bot))
          for filename in label_name_list:
              original=  open(mypath+'/'+filename, "r")
              img_path=os.path.join(outpath, filename.strip('.txt') + ".jpg_grid_" + str(top) + "_" + str(left) + ".jpg")
              new_path=os.path.join(outpath, filename.strip('.txt') + ".jpg_grid_" + str(top) + "_" + str(left) + ".txt")
              new = open(new_path,  "a+")
              lines = original.read().split('\n')
              #print(lines)
              #print(mypath+'/'+filename)
              #print(new_path)
              print(new_path)
              #print(filename)
              for line in lines:
                  #print(line)
                  if line!='':
                      #print(new_path)
                      if isInTheImage(img_path, line):
                          #print(line)
                          #before writing, we need to normalize the new bounding box values
                          newline = convert_coordinates(line, img_path)
                          new.write(newline +'\n')
                          print(newline)
        
        




Output streaming troncato alle ultime 5000 righe.
0 0.22507032306993066 0.5825830334115506 0.14031582952815805 0.13080289193302894
/content/drive/My Drive/microplasticos/labels/train_576/line-03-olympus-10-01-2020.jpg_grid_1728_2112.txt
1 0.1275874859948421 0.05212941162480973 0.126046423135464 0.10425882324961946
1 0.6185401208143076 0.33761663257864005 0.21404109589041156 0.1355593607305935
1 0.5850796840647725 0.21384935074200928 0.309170471841705 0.1926369863013701
/content/drive/My Drive/microplasticos/labels/train_576/line-04-olympus-10-01-2020.jpg_grid_1728_2112.txt
1 0.13550886210965837 0 0.14297385620915054 0
1 0.15060882088174773 0.2898702739197525 0.2689270152505448 0.13276143790849676
1 0.4893620642701515 0.5161332569293149 0.245098039215686 0.1974400871459699
1 0.19647605377027355 0.6318385378842892 0.31998910675381215 0.21786492374727706
1 0.4826247049745821 0.9330516880749202 0.20424836601307164 0.13389662385015955
1 0.7192892251422174 0.25903208023178376 0.1599945533769

In [ ]:
print(grid_height/overlap)

18.0


In [ ]:
l=os.listdir(outpath)
for x in l:
  if 'ar-03' in x:
    i=x.find('.')
    cn= 'tar-03-olympus-10-01-2020' +x[i:] #correcly renaming the file
    print(x, 'renamed to ', cn)
    os.rename(outpath+'/'+x, outpath+'/'+cn)


tar-03-olympus-10-01-2020.jpg_grid_0_0.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_0.txt
tar-03-olympus-10-01-2020.jpg_grid_0_192.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_192.txt
tar-03-olympus-10-01-2020.jpg_grid_0_384.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_384.txt
tar-03-olympus-10-01-2020.jpg_grid_0_576.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_576.txt
tar-03-olympus-10-01-2020.jpg_grid_0_768.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_768.txt
tar-03-olympus-10-01-2020.jpg_grid_0_960.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_960.txt
tar-03-olympus-10-01-2020.jpg_grid_0_1152.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_1152.txt
tar-03-olympus-10-01-2020.jpg_grid_0_1344.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_1344.txt
tar-03-olympus-10-01-2020.jpg_grid_0_1536.txt renamed to  tar-03-olympus-10-01-2020.jpg_grid_0_1536.txt
tar-03-olympus-10-01-2020.jpg_grid_0_1728.txt renamed to  tar-03-olympus-10-01-2

In [ ]:
len(os.listdir(outpath))

6688